In [9]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import time
import zipfile


In [10]:


# Download CoNLL 2003 Dataset
import os
from pathlib import Path
import urllib.request

#if not Path("eng.train").is_file():
#    print("File Not found will downloading it!")
#    url = "https://github.com/patverga/torch-ner-nlp-from-scratch/raw/master/data/conll2003/eng.train"
#    urllib.request.urlretrieve(url, 'eng.train')
#else:
#    print("File already present.")



In [11]:
import sparknlp 

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  4.1.0
Apache Spark version:  3.2.1


In [12]:
nerTagger = NerCrfApproach()\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMinEpochs(1)\
  .setMaxEpochs(1)\
  .setLossEps(1e-3)\
  .setL2(1)\
  .setC0(1250000)\
  .setRandomSeed(0)\
  .setVerbose(0)


In [13]:
from sparknlp.training import CoNLL
conll = CoNLL()
data = conll.readDataset(spark, path='eng_custom.train')

embeddings = WordEmbeddingsModel.pretrained()\
.setOutputCol('embeddings')

ready_data = embeddings.transform(data)

ready_data.show(100)


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        text|            document|            sentence|               token|                 pos|               label|          embeddings|
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        make|[{document, 0, 3,...|[{document, 0, 3,...|[{token, 0, 3, ma...|[{pos, 0, 3, 0, {...|[{named_entity, 0...|[{word_embeddings...|
|         can|[{document, 0, 2,...|[{document, 0, 2,...|[{token, 0, 2, ca...|[{pos, 0, 2, 0, {...|[{named_entity, 0...|[{word_embeddings...|
|          do|[{document, 0, 1,...|[{document, 0, 1,...|[{token, 0, 1, do...|[{pos, 0, 1, 0, {...|[{named_entity, 0...|[{word_embeddings...|
|        read|[{document, 0, 3,...|[{document, 0, 3,...|[

In [14]:
start = time.time()
print("Start fitting")
ner_model = nerTagger.fit(ready_data)
print("Fitting has ended")
print (time.time() - start)


Start fitting
Fitting has ended
0.31803321838378906


In [17]:
ner_model.write().overwrite().save("./pip_wo_embedd/")
import shutil

#shutil.make_archive("./pip_wo_embedd/", 'zip', "/pip_wo_embedd/")

In [24]:

model = NerCrfModel.load("./pip_wo_embedd/")
result = model.annotate("Navigate to kitchen")

print(result['ner'])
print(result['entities'])

AttributeError: 'NerCrfModel' object has no attribute 'annotate'